## HW optional: Word Embeddings

*based on [YSDA](https://github.com/yandexdataschool/nlp_course) materials*

Today we gonna play with word embeddings.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__ if you're running locally, in the selected environment run the following command:

```pip install --upgrade nltk gensim bokeh umap-learn```


In [1]:
import itertools
import string

from nltk.tokenize import WordPunctTokenizer

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

from IPython.display import clear_output

import fasttext

In [2]:
# download the data:
!wget https://yadi.sk/i/BPQrUu1NaTduEw -O ./quora.txt -nc

zsh:1: command not found: wget


In [3]:
data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

### Tokenization (1pt)

a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [4]:
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [5]:
# TASK: lowercase everything and extract tokens with tokenizer.
# data_tok should be a list of lists of tokens for each line in data.

data_tok = list(map(lambda x: tokenizer.tokenize(x.lower()), data))

Let's peek at the result:

In [6]:
' '.join(data_tok[0])

"can i get back with my ex even though she is pregnant with another guy ' s baby ?"

Small check that everything is alright

In [7]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

### Word2Vec (1 pt)

 as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another NLP library that features many vector-based models incuding word2vec.

In [8]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok,
                 vector_size=32,      # embedding vector size
                 min_count=5,      # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [9]:
# now you can get word vectors !
model.get_vector('anything')

array([-2.3881612 ,  3.8410544 , -1.0821643 ,  2.3846953 ,  1.4505802 ,
        3.2288609 ,  0.87300265, -5.153463  ,  1.1331632 ,  2.378453  ,
       -0.2822495 ,  3.0294056 ,  1.809372  ,  0.9509787 ,  1.8516141 ,
       -2.1091504 , -0.40872762, -1.4886427 ,  0.5771764 , -0.63229424,
       -0.00814363,  0.94956565, -0.80441225, -3.0166743 ,  2.937839  ,
       -2.3745322 ,  0.9407915 ,  2.4435062 ,  0.5696545 ,  0.02980437,
       -0.54819965,  1.1424769 ], dtype=float32)

In [10]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

[('rice', 0.9629644155502319),
 ('cheese', 0.9455610513687134),
 ('beans', 0.9311882257461548),
 ('sauce', 0.9293166399002075),
 ('butter', 0.9254921078681946),
 ('banana', 0.9200214743614197),
 ('vodka', 0.9194321036338806),
 ('corn', 0.9149587750434875),
 ('wine', 0.9101898670196533),
 ('chocolate', 0.9069963097572327)]

Show the linear structure in embeddings space with 2-3 examples

In [11]:
model.most_similar(positive=['woman', 'king'], negative=['man']) # expecting queen

[('prince', 0.8741336464881897),
 ('abraham', 0.8670502305030823),
 ('lee', 0.863422691822052),
 ('jackson', 0.8598844408988953),
 ('martin', 0.8440258502960205),
 ('elizabeth', 0.8410847187042236),
 ('queen', 0.8348506689071655),
 ('paul', 0.8347907066345215),
 ('luther', 0.8347095847129822),
 ('stark', 0.8293280005455017)]

In [12]:
model.most_similar(positive=['people', 'science'])

[('teachers', 0.7831771373748779),
 ('opine', 0.7284306883811951),
 ('professors', 0.714881420135498),
 ('others', 0.7104186415672302),
 ('experts', 0.7051659226417542),
 ('passionate', 0.6902500987052917),
 ('students', 0.6832643747329712),
 ('atheists', 0.6780943274497986),
 ('gcse', 0.6758619546890259),
 ('doctors', 0.6650952696800232)]

In [13]:
model.most_similar(positive=['fly', 'vehicle', 'air'], negative=['wheels'])

[('aircraft', 0.8091952204704285),
 ('passenger', 0.7814053297042847),
 ('land', 0.7766246795654297),
 ('tank', 0.7225827574729919),
 ('carrier', 0.7141984105110168),
 ('wire', 0.7124768495559692),
 ('outlet', 0.7120674848556519),
 ('flight', 0.711766242980957),
 ('bus', 0.71099454164505),
 ('operate', 0.6938575506210327)]

### Glove + PCA (1 pt)

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [14]:
import gensim.downloader as api

model = api.load('glove-twitter-25')

In [15]:
words = sorted(model.key_to_index, 
               key=lambda word_key: model.get_vecattr(word_key, "count"),
               reverse=True)[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [16]:
# for each word, compute it's vector with model
word_vectors = np.array([model[word] for word in words])

In [17]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 25)
assert np.isfinite(word_vectors).all()

In [18]:
word_vectors.shape

(1000, 25)

Use PCA to decrease embeddings dimension so that we will be able to draw them.

In [19]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
pca = PCA(2)
scaler = StandardScaler()
# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
word_vectors_pca = pca.fit_transform(word_vectors)
word_vectors_pca = scaler.fit_transform(word_vectors_pca)

In [20]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

Function to draw embeddings

In [21]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [22]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

figure(id='p1004', ...)

<font color=5e60ce>We could see a big English, Emoji, Spanish, Arabic and Indonesian language clusters. </font>

### Phrase Embeddings (1 pt)

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.


In [23]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros

    phrase = phrase.lower()
    tokens = tokenizer.tokenize(phrase)
    vector = np.zeros(word_vectors.shape[1])
    
    cnt = 0
    for token in tokens:
        if token in model.key_to_index:
            vector += model[token]
            cnt += 1
    return vector / cnt if cnt > 0 else vector

In [24]:
get_phrase_embedding(data[402687])

array([-0.18205   ,  0.30953575,  0.20861092,  0.07982158, -0.22565517,
       -0.3300175 ,  1.24957835,  0.13134292, -0.33788875,  0.06196944,
       -0.231793  ,  0.09389218, -4.96854999, -0.23611419, -0.32609667,
       -0.092073  ,  0.4407505 , -0.7541375 , -0.5389092 , -0.184752  ,
        0.07867809,  0.20018657, -0.16202375,  0.303757  , -0.41255667])

In [25]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

In [26]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases and turn them to numpy array
phrase_vectors = np.asarray([get_phrase_embedding(x) for x in chosen_phrases]) # YOUR CODE

In [27]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [28]:
phrase_vectors[0]

array([-0.20129163,  0.28008468,  0.20248146, -0.06290309, -0.13497116,
        0.00861136,  1.07697579,  0.28034084, -0.48734121,  0.22369644,
       -0.21113858,  0.33287615, -4.84304211, -0.35945611, -0.11689068,
        0.06346284,  0.25226158, -0.48065658, -0.50974858, -0.159933  ,
        0.17304395,  0.26804752,  0.10165837,  0.13623684, -0.20279761])

In [29]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize
from sklearn.manifold import TSNE

tsne = TSNE(perplexity=80)
pca_many_dims = PCA(n_components=10)
phrase_vectors_2d = pca_many_dims.fit_transform(phrase_vectors)
phrase_vectors_2d = tsne.fit_transform(phrase_vectors_2d)
phrase_vectors_2d = scaler.fit_transform(phrase_vectors_2d)

In [30]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

figure(id='p1053', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [31]:
# filter all sentences that include at least one NONPRINTABLE char
# compute vector embedding for all remaining lines in data

printable_set = set(string.printable)

data_subset = [x for x in data if set(x).issubset(printable_set)]

In [32]:
data_vectors = np.vstack([get_phrase_embedding(x) for x in data])

norms = np.linalg.norm(data_vectors, axis=1)
norms[norms == 0] = 0.00001

In [33]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    assert len(norms) == len(data_vectors)
    
    query_emb = get_phrase_embedding(query)
    query_norm = np.linalg.norm(query_emb)
    
    cosines = []
    for i in range(len(norms)):
        cosines.append(np.dot(query_emb, data_vectors[i]) / norms[i] / query_norm)
    args = np.flip(np.argsort(cosines))[:k]
    nearest = [data[args[i]] for i in range(k)]
    return nearest

In [34]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'

How do I get to the dark web?
What universal remote do I need and how do I set it up to a Blaupunkt TV?
How do I connect the ASUS_T00Q to my PC?
How do you print the gridlines in Excel 2010?
How do you print the gridlines in Excel 2007?
How do you print the gridlines in Excel 2003?
I would like to create a new website. What do I have to do?
How do I get the new Neko Atsume wallpapers? How do they work?
I want to experience the 4G network. Do I need to change my SIM card from 3G to 4G?
What do I have to do to sell my photography?



In [35]:
find_nearest('i love coffee so much')

['How can I make my baby sleep?\n',
 'How can I forget the girl I love so much?\n',
 'How can I make my life beautiful?\n',
 'How can I make myself feel beautiful?\n',
 'How can I find my love?\n',
 'Why do I feel so warm and fuzzy when in love?\n',
 'Is it good to eat too much before bed?\n',
 'Am I ever going to find my true love?\n',
 'How much do I sleep so I remember my dreams?\n',
 'Where can I find my true love?\n']

### Phrase Embeddings with TfIdf (2 pt, bonus)

Improve the above __get_phrase_embedding__ function to use TF-IDF as vector weights.

1) train sklearn tf-idf implementation
2) update function *get_phrase_embedding*
3) compare new and old versions using *find_nearest* on 2-3 examples 

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [37]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data)

In [38]:
tfidf_matrix.shape

(537272, 86108)

In [39]:
vectorizer.get_feature_names_out()[10000:10005]

array(['baherain', 'bahia', 'bahira', 'bahr', 'bahrain'], dtype=object)

In [40]:
tfidf_data = pd.DataFrame(vectorizer.idf_, index=vectorizer.get_feature_names_out(), columns=["Weight"])
tfidf_data.sort_values(by="Weight")

,Weight
what,1.944794
the,1.971096
is,2.121480
how,2.394489
in,2.453181
...,...
peacoat,13.501114
enteropathic,13.501114
enteroctopus,13.501114
peary,13.501114


In [41]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros

    phrase = phrase.lower()
    tokens = tokenizer.tokenize(phrase)
    vector = np.zeros(word_vectors.shape[1])
    
    for token in tokens:
        weights_sum = 0
        if token in model.key_to_index and token in tfidf_data['Weight'].index:
            vector += model[token] * tfidf_data['Weight'][token]
            weights_sum += tfidf_data['Weight'][token]
    return vector / weights_sum if weights_sum > 0 else vector

In [42]:
find_nearest(query="Why don't i ask a question myself?", k=10)

['How do I respond if someone calls me dumb?\n',
 'How do I respond when someone gives me a compliment?\n',
 'How can I ask someone a personal question on Quora?\n',
 'How can I stop Quora asking me to answer a particular question?\n',
 'How do I say when people ask me rude question?\n',
 'How do you delete a question you asked on Quora\n',
 "What are some reasons people don't respond to text messages?\n",
 'How can I ask someone to answer a question on Quora?\n',
 'How do you leave a voicemail without calling that person?\n',
 'Can I ask a question and answer it myself on Quora?\n']

### FastText (2 pt, bonus)

Implement search enginge using [FastText](https://fasttext.cc/docs/en/python-module.html) embeddings. 

1) train fasttext embeddings on all data
2) update *get_phrase_embeddings* function
3) implement *find_nearest* function with fasttext embeddings
4) compare results with glove version from the main task

In [43]:
model = fasttext.train_unsupervised('quora.txt')

Read 6M words
Number of words:  44643
Number of labels: 0
Progress: 100.0% words/sec/thread:  181834 lr:  0.000000 avg.loss:  1.894151 ETA:   0h 0m 0s 1.894151 ETA:   0h 0m 0s


In [44]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros

    phrase = phrase.lower()
    tokens = tokenizer.tokenize(phrase)
    vector = np.zeros(model.dim)
    
    for token in tokens:
        weights_sum = 0
        if token in model.words and token in tfidf_data['Weight'].index:
            vector += model[token] * tfidf_data['Weight'][token]
            weights_sum += tfidf_data['Weight'][token]
    return vector / weights_sum if weights_sum > 0 else vector

In [45]:
data_vectors = np.vstack([get_phrase_embedding(x) for x in data])

norms = np.linalg.norm(data_vectors, axis=1)
norms[norms == 0] = 0.00001

In [46]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    assert len(norms) == len(data_vectors)
    
    query_emb = get_phrase_embedding(query)
    query_norm = np.linalg.norm(query_emb)
    
    cosines = []
    for i in range(len(norms)):
        cosines.append(np.dot(query_emb, data_vectors[i]) / norms[i] / query_norm)
    args = np.flip(np.argsort(cosines))[:k]
    nearest = [data[args[i]] for i in range(k)]
    return nearest

In [47]:
find_nearest(query="How do i enter the matrix?", k=10)

['How do we get inversion of 5*5 matrix in Verilog?\n',
 'What is an adjacency matrix?\n',
 'Who is Dan in the Matrix?\n',
 'Are we in the matrix?\n',
 'What is an augmented matrix and how do we find its rank?\n',
 'Are we really in the matrix?\n',
 'Do you live in the matrix? Why?\n',
 'Use of matrix in busines?\n',
 'Are we in a matrix?\n',
 'What is The Matrix about?\n']

In [48]:
find_nearest('i love coffee so much')

['Why is coffee so addicting?\n',
 'Do you drink coffee? What are the reasons why you think people enjoy drinking coffee so much?\n',
 'Can drinking too much coffee actually make you more tired?\n',
 'Why do so many people enjoy drinking coffee?\n',
 'Why do Russians drink so much Vodka?\n',
 'How do I make coffee?\n',
 'Does coffee make you hungry?\n',
 'How much coffee should I drink?\n',
 'Why does coffee make me thirsty?\n',
 'Why do I feel so good after drinking decaf coffee?\n']

In [49]:
find_nearest(query="Why don't i ask a question myself?", k=10)

['Can I ask a question and answer it myself on Quora?\n',
 'What questions should I/everybody ask myself?\n',
 'Can you ask me a question?\n',
 'How do I ask myself the right question to know myself?\n',
 'How do I ask a question intelligently?\n',
 'How do I ask a question?\n',
 'How do you ask a question?\n',
 'What questions should I be asking about myself?\n',
 'Why do I ask this question?\n',
 'How do you ask a question in Quora?\n']

In [50]:
model.dim

100